In [2]:
import pandas as pd
import altair as alt
import numpy as np
from scipy.stats import gaussian_kde
from sklearn.neighbors import KernelDensity

In [3]:
data = pd.read_csv('Mother Jones - Mass Shootings Database, 1982 - 2023 - Sheet1.csv', parse_dates=['date'])
data = data.loc[data['fatalities']>=4]
data['deltaT'] = (data['date']-data['date'].min()).dt.days
kde = gaussian_kde(data['deltaT'], weights=data['fatalities'])
kde.set_bandwidth(0.05)
samples = np.linspace(0, data['deltaT'].max(), 1000)
density = kde(samples)
density_frame = pd.DataFrame({
    'delta_time':samples,
    'density':density
})
density_frame['date'] = pd.to_timedelta(density_frame['delta_time'],'d') + data['date'].min()

In [4]:
from datetime import datetime, timedelta
cw = 700

base = alt.Chart(density_frame, title='Density of Mass Shooting Fatalities').mark_area().encode(
    alt.X('date:T', axis=alt.Axis(title='Date')),
    alt.Y('density:Q', axis=alt.Axis(title='Density'))
    # y='fatalities:Q'
    # size='fatalities:Q'
)
rules = alt.Chart(pd.DataFrame({
    'Date':[datetime(1994,9,13),
            datetime(2004,9,13),
            datetime(2012,11,7)],
    'Event':['AWB signed','AWB sunset', 'Obama re-elected']
})).mark_rule(strokeWidth=5).encode(
    x='Date:T',
    color=alt.Color('Event:N')
)
points = alt.Chart(data).mark_circle().encode(
    alt.X('date:T', axis=alt.Axis(title='Date')),
    size='fatalities:Q'
).properties(
    width=cw
) + rules
chart = (base + rules).properties(height=300, width=cw)
chart = alt.vconcat(chart, points)
chart

/Users/sam/anaconda3/envs/infovis/lib/python3.10/site-packages/altair/utils/core.py:283: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)